# Contextual question answering

## Objectives (8 points)

1. Get acquainted with the [Simple legal questions dataset](https://github.com/apohllo/simple-legal-questions-pl) (you need to send your github login to gain access to the dataset).

3. The legal questions dataset is your **test dataset**.

In [2]:
import pandas as pd

legal_dataset = pd.read_csv('./simple-legal-questions-pl-main/combined.csv')

In [3]:
print(legal_dataset.head())

         passage_id                                           question  \
0  1997_553_345.txt  Czy żołnierz, który dopuszcza się czynnej napa...   
1   2004_177_21.txt         Z ilu osób składa się komisja przetargowa?   
2  1996_465_111.txt  Do jakiej wysokości za zobowiązania spółki odp...   
3   1994_591_35.txt  Kiedy ustala się wartość majątku obrotowego, k...   
4  2001_1441_74.txt  Jakiej karze podlega armator, który wykonuje r...   

                                             passage  has_answer  year  \
0  Art. 345. § 1. Żołnierz, który dopuszcza sie...        True  1997   
1  Art. 21. 1. Członków komisji przetargowej pow...        True  2004   
2  Art. 111. Komandytariusz odpowiada za zobowią...        True  1996   
3  Art. 35. 1. Wartość rzeczowych składników m...        True  1994   
4  Art. 74. 1. Armator, który wykonuje rybołóws...        True  2001   

   position  art  
0       553  345  
1       177   21  
2       465  111  
3       591   35  
4      1441   7

4. [PoQuAD](https://huggingface.co/datasets/clarin-pl/poquad) is your **train and validation dataset** (use the splits from the repo).

In [5]:
from datasets import load_dataset
import pandas as pd
import json

with open('poquad-train.json', 'r') as file:
    poquad_train = json.load(file)

with open('poquad-dev.json', 'r') as file:
    poquad_dev = json.load(file)

test_set = pd.read_csv("./simple-legal-questions-pl-main/combined.csv")

/home/adrian/dev/nlp/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
print(poquad_train['data'][0])

{'id': 7609, 'title': 'Konfederacja polsko-czechosłowacka', 'summary': 'Konfederacja polsko-czechosłowacka – koncepcja polityczna z okresu II wojny światowej propagowana przez Rząd RP na uchodźstwie. Idea ta, wspierana przez rząd Wielkiej Brytanii, nawiązywała do przedwojennych planów ustanowienia bloku państw „Międzymorza”. Początkowo częściami składowymi konfederacji miały być Polska, Czechosłowacja i Węgry.\nW okresie II wojny światowej rządy niektórych państw europejskich na uchodźstwie dokonywały prób jednoczenia się w związki, które miały dać im w przyszłości gwarancję bezpieczeństwa, trwałości politycznej i gospodarczej. Odżywały koncepcje federacyjne z lat 20. i 30. XX w., które w nowych warunkach miały szansę realizacji. Wiele projektów federalistycznych można odnaleźć w programach polskich stronnictw politycznych w okresie II wojny światowej. Takie próby były dokonywane przez Polskę, Czechosłowację, Grecję i Jugosławię. Projekt federacji (konfederacji) polsko-czechosłowackiej

In [7]:
print(poquad_dev['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['generative_answer'])

kompilacją poglądów różnych rabinów na określony temat


In [8]:
print(test_set.head())

         passage_id                                           question  \
0  1997_553_345.txt  Czy żołnierz, który dopuszcza się czynnej napa...   
1   2004_177_21.txt         Z ilu osób składa się komisja przetargowa?   
2  1996_465_111.txt  Do jakiej wysokości za zobowiązania spółki odp...   
3   1994_591_35.txt  Kiedy ustala się wartość majątku obrotowego, k...   
4  2001_1441_74.txt  Jakiej karze podlega armator, który wykonuje r...   

                                             passage  has_answer  year  \
0  Art. 345. § 1. Żołnierz, który dopuszcza sie...        True  1997   
1  Art. 21. 1. Członków komisji przetargowej pow...        True  2004   
2  Art. 111. Komandytariusz odpowiada za zobowią...        True  1996   
3  Art. 35. 1. Wartość rzeczowych składników m...        True  1994   
4  Art. 74. 1. Armator, który wykonuje rybołóws...        True  2001   

   position  art  
0       553  345  
1       177   21  
2       465  111  
3       591   35  
4      1441   7

5. **Warning** PoQuAD has a python API compatible with the `datasets` library, but it only provides the **extractive answers**, even
   though the abstractive answers are available in the JSON files. So you have to read the JSON files directly.

7. Train a neural model able to answer the legal questions. Make sure you are using a machine
   with a GPU, since training the model on CPU will be very long. 
   The training should include at least 3 epochs (depending on the size of the training set you are using). 
   As the pre-trained models you can use (or any other model that is able to perform abstractive Question Answering):
   * [plT5-base](https://huggingface.co/allegro/plt5-base)
   * [plT5-large](https://huggingface.co/allegro/plt5-large)

In [9]:
from transformers import TrainingArguments, AutoModelForSeq2SeqLM, AutoTokenizer, Trainer
import numpy as np
import random
from tqdm import tqdm

In [97]:
model_name = "allegro/plt5-base"
pl_tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [166]:
def tokenize_function(example):
    return pl_tokenizer(example, return_tensors="pt", padding=True, truncation=True, max_length=1000)['input_ids'].squeeze()

def tokenize_example(example):
    input = pl_tokenizer(example['text'], return_tensors="pt", padding=True, truncation=True, max_length=1000)

    with pl_tokenizer.as_target_tokenizer():
        target = pl_tokenizer(example['label'], return_tensors="pt", padding=True, truncation=True, max_length=1000)

    return {'input_ids': input['input_ids'].squeeze(), 'label': target['input_ids'].squeeze()}

def prepare_data(json_poquad):
    prepared_data = []
    for data in tqdm(json_poquad['data']):
        for paragraph in data['paragraphs']:
            context = paragraph['context']
            for qas in paragraph['qas']:
                question = qas['question']
                ans_keys = ['answers', 'plausible_answers']
                for key in ans_keys:
                    if key in qas:
                        for answer in qas[key]:
                            generative_answer = answer['generative_answer']
                            prepared_data.append({"text": f"question: {question} context: {context}", "label": generative_answer})
    return prepared_data


In [167]:
from datasets import Dataset

ds_train = Dataset.from_list(prepare_data(poquad_train))
ds_test = Dataset.from_list(prepare_data(poquad_dev))

100%|██████████| 1402/1402 [00:00<00:00, 117439.17it/s]


In [168]:
ds_train = ds_train.map(tokenize_example, batched=True)

Map:   0%|          | 0/56618 [00:00<?, ? examples/s]

Map: 100%|██████████| 56618/56618 [00:33<00:00, 1696.47 examples/s]


In [169]:
ds_test = ds_test.map(tokenize_example, batched=True)

Map:   0%|          | 0/7060 [00:00<?, ? examples/s]

Map: 100%|██████████| 7060/7060 [00:04<00:00, 1473.57 examples/s]


In [170]:
df_train = prepare_data(poquad_train)
df_test = prepare_data(poquad_dev)
print(len(df_train), len(df_test))

100%|██████████| 1402/1402 [00:00<00:00, 107889.59it/s]

56618 7060


8. If you have problems training the model, you can use [apohllo/plt5-base-poquad](https://huggingface.co/apohllo/plt5-base-poquad) which was trained on PoQuAD. **This will result in  subtraction of 2 points**. 

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("apohllo/plt5-base-poquad")
model = AutoModelForSeq2SeqLM.from_pretrained("apohllo/plt5-base-poquad")

In [43]:
manual_answers_improved= [
    'Żołnierz, który dopuszcza się czynnej napaści na przełożonego, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.',
    'z co najmniej trzech osób',
    'do wysokości sumy komandytowej',
    'nie później niż na dzień bilansowy',
    'pieniężnej do wysokości 1 000 000 złotych',
    'organizacji międzynarodowych, przedstawicielstw dyplomatycznych, urzędów konsularnych oraz członków personelu tych przedstawicielstw i urzędów, a także innych osób zrównanych z nimi na podstawie ustaw, umów lub zwyczajów międzynarodowych, jeżeli nie są obywatelami polskimi i nie mają stałego miejsca pobytu na terytorium kraju. ',
    'są obowiązani przedstawić się',
    'jeden',
    'Wykonując czynności operacyjno-rozpoznawcze, żołnierze Żandarmerii Wojskowej mogą posługiwać się dokumentami, które uniemożliwiają ustalenie danych pozwalających na identyfikację oraz środków, którymi posługują się, wykonując te czynności.',
    'W celu znalezienia i zatrzymania przedmiotów podlegających oględzinom lub mogących stanowić dowód rzeczowy',
    'na warunkach uzgodnionych w umowie',
    'Karze pieniężnej',
    '1) wicewojewodowie, 2) dyrektor generalny urzędu wojewódzkiego, 3) komendant wojewódzki Policji, 4) komendant wojewódzki Państwowej Straży Pożarnej, 5) inne osoby wymienione w statucie urzędu wojewódzkiego.',
    'Poręczenia są terminowe',
    'Funkcjonariusz podlega corocznemu opiniowaniu służbowemu'
]

In [48]:
tokenized_legal_questions = []
idx = 0
for id, row in legal_dataset.iterrows():
    tokenized_legal_questions.append([tokenizer(f"question: {row['question']} context: {row['passage']}", return_tensors="pt", padding=True, truncation=True, max_length=512), row['question'], manual_answers_improved[idx]])
    idx += 1
    if idx == 15: break


In [27]:
print(tokenized_legal_questions[0])

[{'input_ids': tensor([[  277, 18616,  4767,   291,   863,  8390,   261,   345, 11748,   271,
         26942, 37351,   266,   418, 10000,   323,  8479, 21534,  5493,   267,
          3543,   373, 12634, 22091,   399,   291,  3470,   259,   361,  1177,
           259, 32556,   959, 17124,   261,   345, 11748,   271, 26942, 37351,
           266,   418, 10000,   323,   261,  8479, 21534, 12647, 10817,  1263,
          5493,   267,  3543,   272,   346,  1193, 32556,  1161,  2237, 11627,
         11748,   271, 26942, 37351,   260,  1019,   265,  2062,  2523,   299,
           418, 10000,   323,  7947, 27133,  1263,  1746,   265,   877,  8390,
           320,   372,   260,  6267, 18438,  2186,   261,  8479, 21534,  5493,
           267,  3543,   289,   423,  1389,   272,   346,  2846, 32556,  1193,
          2237, 11627, 25583, 27697,   260, 32556,   333,   372,   334,  5137,
          2629,   261,  3151,  2789,   372,  2712,  2155, 11033,   323, 22210,
           261,  8479, 21534,  5493, 

9. Report the obtained performance of the model (in the form of a table). The report should include *exact match* and *F1 score* 
   for the tokens appearing both in the reference and the predicted answer.

In [28]:
import string

def simplify_text(prediction, label):
    prediction = prediction.lower()
    label = label.lower()

    prediction = "".join(character for character in prediction if character not in set(string.punctuation))
    label = "".join(character for character in label if character not in set(string.punctuation))

    prediction = " ".join(prediction.split())
    label = " ".join(label.split())
    return prediction, label 

def exact_match(prediction, label):
    prediction, label = simplify_text(prediction, label)
    return prediction == label

def f1(prediction, label):
    prediction, label = simplify_text(prediction, label)
    prediction = prediction.split()
    label = label.split()
    intersection = set(prediction).intersection(set(label))
    
    if len(intersection) == 0: return 0
    
    precision = len(intersection) / len(prediction)
    recall = len(intersection) / len(label)
    
    return 2 * (precision * recall) / (precision + recall)

In [49]:
from prettytable import PrettyTable 
exact_match_outcomes = []
f1_outcomes = []
table_with_scores = PrettyTable(['Question', 'exact_match', 'f1'])

for idx, (tokenized_question, question, label) in enumerate(tokenized_legal_questions):
    output = model.generate(**tokenized_question)
    prediction = tokenizer.decode(output[0], skip_special_tokens=True)
    em_s = exact_match(prediction, label)
    f1_s = f1(prediction, label)
    exact_match_outcomes.append(em_s)
    f1_outcomes.append(f1_s)
    table_with_scores.add_row([question, em_s, round(f1_s, 3)])

print(table_with_scores)

+---------------------------------------------------------------------------------------------------------------------------------+-------------+-------+
|                                                             Question                                                            | exact_match |   f1  |
+---------------------------------------------------------------------------------------------------------------------------------+-------------+-------+
|              Czy żołnierz, który dopuszcza się czynnej napaści na przełożonego podlega karze pozbawienia wolności?              |    False    |   0   |
|                                            Z ilu osób składa się komisja przetargowa?                                           |    False    | 0.333 |
|                               Do jakiej wysokości za zobowiązania spółki odpowiada komandytariusz?                              |    False    | 0.667 |
|                          Kiedy ustala się wartość majątku obrotowego, któr

10. Report the best results obtained on the validation dataset and the corresponding results on your test dataset. The results on the 
   test set have to be obtained for the model that yield the best result on the validation dataset.

In [36]:
table_with_scores_poquad = PrettyTable(['Question', 'exact_match', 'f1'])
cnt = 0
maxi = 15
for data in tqdm(poquad_dev['data']):
    for paragraph in data['paragraphs']:
        context = paragraph['context']
        for qas in paragraph['qas']:
            question = qas['question']
            ans_keys = ['answers', 'plausible_answers']
            for key in ans_keys:
                if key in qas:
                    for answer in qas[key]:
                        label = answer['generative_answer']

                        tokenized_question = tokenizer(f"question: {question} context: {context}", return_tensors="pt", padding=True, truncation=True, max_length=512)
                        output = model.generate(**tokenized_question)
                        prediction = tokenizer.decode(output[0], skip_special_tokens=True)
                        
                        em_s = exact_match(prediction, label)
                        f1_s = f1(prediction, label)
                        exact_match_outcomes.append(em_s)
                        f1_outcomes.append(f1_s)
                        cnt += 1
                        table_with_scores_poquad.add_row([question, em_s, round(f1_s, 3)])
                        if cnt == maxi: break
                if cnt == maxi: break
            if cnt == maxi: break
        if cnt == maxi: break
    if cnt == maxi: break

print(table_with_scores_poquad)

  0%|          | 3/1402 [00:20<2:40:13,  6.87s/it]

+--------------------------------------------------------------------------------------------------+-------------+-------+
|                                             Question                                             | exact_match |   f1  |
+--------------------------------------------------------------------------------------------------+-------------+-------+
|                                    Czym są pisma rabiniczne?                                     |     True    |  1.0  |
|                    Z ilu komponentów składała się Tora przekazana Mojżeszowi?                    |     True    |  1.0  |
|                     W jakich formach występowała Tora przekazana Mojżeszowi?                     |    False    | 0.364 |
|                            W jakiej formie przekazana została Miszna?                            |    False    |   0   |
|                                        Kto napisał Torę?                                         |    False    |   0   |
|               

11. Generate, report and analyze the answers for at least 10 questions provided by the best model on you test dataset.

In [54]:
answers_10 = [
    'karze pieniężnej do wysokości 1 000 000 złotych',
    'W celu znalezienia i zatrzymania przedmiotów podlegających oględzinom lub mogących stanowić dowód rzeczowy',
    'Funkcjonariusz podlega corocznemu opiniowaniu służbowemu',
    'Budżet państwa jest rocznym planem dochodów i wydatków oraz przychodów i rozchodów',
    'Do przestępstw skarbowych nie mają zastosowania przepisy części ogólnej Kodeksu karnego',
    'na akcje o równej wartości nominalnej',
    'Kontroler jest obowiązany zachować w tajemnicy informacje, które uzyskał w związku z wykonywaniem czynności służbowych',
    'na polecenie sądu lub prokuratora Policja',
    '2. Rejestr grup zawiera: 1) nazwę i siedzibę grupy, 2) datę wydania decyzji administracyjnej, o której mowa w art. 7 ust. 1, 3) nazwę produktu lub grupy produktów, ze względu na które grupa otrzymała decyzję, 4) dane osób upoważnionych do reprezentowania grupy zgodnie z jej aktem założycielskim.',
    ''
]

idx_mini = 0
idx = 0
for id, row in legal_dataset.iterrows():
    idx += 1
    if idx%5 != 0: continue

    tokenized = tokenizer(f"question: {row['question']} context: {row['passage']}", return_tensors="pt", padding=True, truncation=True, max_length=512)
    output = model.generate(**tokenized)
    prediction = tokenizer.decode(output[0], skip_special_tokens=True)
    em_s = exact_match(prediction, answers_10[idx_mini])
    f1_s = f1(prediction, answers_10[idx_mini])
    print(f"Question: {row['question']}")
    print(f"Answer: {answers_10[idx_mini]}")
    print(f"Generated answer: {prediction}")
    print(f"Exact match score: {em_s}")
    print(f"F1 score: {f1_s}")
    print(f"============================================================================================================================================")

    idx_mini += 1
    if idx == 50: break

Question: Jakiej karze podlega armator, który wykonuje rybołówstwo morskie w polskich obszarach morskich, z naruszeniem przepisów ustawy?
Answer: karze pieniężnej do wysokości 1 000 000 złotych
Generated answer: pieniężnej do wysokości 1 000 000 złotych
Exact match score: False
F1 score: 0.5333333333333333
Question: W jakim przypadku policja może dokonać przeszukania pomieszczeń w domu?
Answer: W celu znalezienia i zatrzymania przedmiotów podlegających oględzinom lub mogących stanowić dowód rzeczowy
Generated answer: jeżeli istnieją uzasadnione podstawy do przypuszczenia, że przedmioty te lub dowody tam się znajdują
Exact match score: False
F1 score: 0.07407407407407408
Question: Co jaki okres funkcjonariusze podlegają opiniowaniu służbowemu?
Answer: Funkcjonariusz podlega corocznemu opiniowaniu służbowemu
Generated answer: coroczny
Exact match score: False
F1 score: 0
Question: Czym jest budżet państwa?
Answer: Budżet państwa jest rocznym planem dochodów i wydatków ora

## Questions (2 points)

Training didn't work for me, so I used prepared models.

1. Does the performance on the validation dataset reflects the performance on your test set?\
   **No, the performance on the validation dataset is much better compared to results obtained using the test set**
2. What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason
   for that?\
   **It depends. To some degree, they are satisfying.
      After analysis, I came to the conclusion that there are different kinds of results:
      1. Good but very concise. (exact match cannot match it)
      2. Good and exact. (exact match works here)
      3. False, because a bad snippet of the sentence was extracted. (none of the score methods works here)
      4. False, because they constitute only a part of a proper answer.(f1 score some points here)
      Also sometimes provided articles (e.g. 'Art. 1, ust. 7') have useful information that model cannot capture.

      I think the reasons that the model becomes weakened are:
      1. too complex sentences
      2. The model was trained on poquad dataset and wasn't prepared for this type of content (legal language)
      3. incorporated articles, paragraphs and attachments
      **
3. Why extractive question answering is not well suited for inflectional languages?\
**Because to answer the question we have to change part of a word to fit it properly into the answer that it can be e.g. grammatically correct.**